In [ ]:
from pytorch_optim_training_manager import train_manager
import torch
import torchvision
from datasets import load_dataset
import torchvision.transforms as transforms
import models
import os
import re
from torch.utils.data import Dataset, DataLoader
from ucimlrepo import fetch_ucirepo 
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from custom_datasets import WineDataset, LSTMDataset
from utils import get_LSTM_dataloaders, isEnglish, lowerCase, build_vocabulary

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
print(device)

In [ ]:
os.environ["WANDB_DISABLED"] = "true"
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
# set seeds for reproducibility
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)

P1

In [ ]:
# fetch dataset 
wine_quality = fetch_ucirepo(id=186) 
  
# data (as pandas dataframes) 
x = wine_quality.data.features 
y = wine_quality.data.targets 

In [ ]:
# get number of classes
classes = len(set(y['quality']))

In [ ]:
# split data into train and validation split
# fix random state for reproducibility
data_train, data_validation = train_test_split(x.join(y), random_state=104, test_size=0.25, shuffle=True)

In [ ]:
# normalize the data
scaler = MinMaxScaler()
columns_to_normalize = data_train.columns.tolist()[:-1]

data_train[columns_to_normalize] = scaler.fit_transform(data_train[columns_to_normalize])
data_validation[columns_to_normalize] = scaler.transform(data_validation[columns_to_normalize])

In [ ]:
# create dataloaders
dataset_train = WineDataset(data_train, y['quality'])
dataset_validation = WineDataset(data_validation, y['quality'])
training_loader = DataLoader(dataset_train,batch_size=128,shuffle=True)
validation_loader = DataLoader(dataset_validation,batch_size=128,shuffle=False)

In [ ]:
model = models.Simple_Net(n_classes=classes)

In [ ]:
# save initial state dict of the model
torch.save(model.state_dict(), 'models/Simple_Net.pt')

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()

restart here when evaluating multiple optimizers 

In [ ]:
# load the state dict to restart from base model
model.load_state_dict(torch.load('models/Simple_Net.pt'))

In [ ]:
# choose optimizer for this run
optimizer = torch.optim.RMSprop(model.parameters())

In [ ]:
# define the manager
manager = train_manager(model, loss_fn, optimizer,training_loader,validation_loader,device=device)

In [ ]:
# train the model and return train loss and validation accuracy
losses = manager.train(20, verbose=True, eval_all_epochs=True, eval_mode='accuracy', is_wine=True)

In [ ]:
# get max validation accuracy
print(torch.max(torch.tensor(losses[1])))

In [ ]:
# save accuracy of each epoch to file
torch.save(torch.tensor(losses[1]), 'results/RMSprop_P1.pt')

P2

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

In [ ]:
training_set = torchvision.datasets.FashionMNIST('./data', train=True, transform=transform, download=True)
validation_set = torchvision.datasets.FashionMNIST('./data', train=False, transform=transform, download=True)

In [ ]:
training_loader = torch.utils.data.DataLoader(training_set, batch_size=128, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=128, shuffle=False)

In [ ]:
model = models.CNN_Simple()

In [ ]:
# save initial state dict of the model
torch.save(model.state_dict(), 'models/CNN_Simple.pt')

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()

start again here if evaluating for multiple optimizers

In [ ]:
# load the state dict to restart from base model
model.load_state_dict(torch.load('models/CNN_Simple.pt'))

In [ ]:
# choose optimizer for this run
optimizer = torch.optim.RMSprop(model.parameters())

In [ ]:
manager = train_manager(model, loss_fn, optimizer,training_loader,validation_loader,device=device)

In [ ]:
# train the model and return train loss and validation accuracy
losses = manager.train(20, verbose=True, eval_all_epochs=True, eval_mode='accuracy')

In [ ]:
# get max validation accuracy
print(torch.max(torch.tensor(losses[1])))

In [ ]:
torch.save(torch.tensor(losses[1]), 'results/RMSprop_P2.pt')

P3

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

In [ ]:
training_set = torchvision.datasets.FashionMNIST('./data', train=True, transform=transform, download=True)
validation_set = torchvision.datasets.FashionMNIST('./data', train=False, transform=transform, download=True)
training_loader = torch.utils.data.DataLoader(training_set, batch_size=128, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=128, shuffle=False)

In [ ]:
model = models.VAE()

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
# save initial state dict of the model
torch.save(model.state_dict(), 'models/VAE.pt')

restart here if evaluating for multiple optimizers

In [ ]:
# load the state dict to restart from base model
model.load_state_dict(torch.load('models/VAE.pt'))

In [ ]:
# choose optimizer for this run
optimizer = torch.optim.RMSprop(model.parameters())

In [ ]:
manager = train_manager(model, loss_fn, optimizer, training_loader, validation_loader, device=device)

In [ ]:
# train the model and return train loss and validation loss
losses = manager.train(20, verbose=True, eval_all_epochs=True, eval_mode='loss')

In [ ]:
# get min validation loss
print(torch.min(torch.tensor(losses[1])))

In [ ]:
torch.save(torch.tensor(losses[1]), 'results/RMSprop_P3.pt')

P4

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5074,0.4867,0.4411),(0.2011,0.1987,0.2025))]
)

In [ ]:
training_set = torchvision.datasets.CIFAR100('./data', train=True, transform=transform, download=True)
validation_set = torchvision.datasets.CIFAR100('./data', train=False, transform=transform, download=True)
training_loader = torch.utils.data.DataLoader(training_set, batch_size=256, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=256, shuffle=False)

In [ ]:
model = models.All_CNN_C()

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
# save initial state dict of the model
torch.save(model.state_dict(), 'models/All_CNN_C.pt')

restart here if evaluating multiple optimizers

In [ ]:
# load the state dict to restart from base model
model.load_state_dict(torch.load('models/All_CNN_C.pt'))

In [ ]:
# choose optimizer for this run
optimizer = torch.optim.RMSprop(model.parameters())

In [ ]:
manager = train_manager(model, loss_fn, optimizer, training_loader, validation_loader, device=device)

In [ ]:
# train the model and return train loss and validation accuracy
losses = manager.train(20, verbose=True, eval_all_epochs=True, eval_mode='accuracy')

In [ ]:
# get max validation accuracy
print(torch.max(torch.tensor(losses[1])))

In [ ]:
torch.save(torch.tensor(losses[1]), 'results/RMSprop_P4.pt')

P5

In [ ]:
# get wikitext dataset from datasets library
dataset = load_dataset("wikitext", 'wikitext-2-v1', split='train')

In [ ]:
# clean the data
dataset = dataset.filter(lambda x: 100 <= len(x['text'].split()) <= 128)
dataset = dataset.filter(lambda x: not re.match(" = .* = \n", x['text']))
dataset = dataset.filter(lambda x: isEnglish(x['text']))
dataset = dataset.map(lambda x: lowerCase(x))

In [ ]:
# build wikitext vocabulary for the dataset
wikitext_dataset, token_freq_dict = build_vocabulary(dataset, min_freq=5, unk_token='<unk>')

In [ ]:
MAX_SEQ_LENGTH = 128
lstm_dataset = LSTMDataset(dataset=wikitext_dataset,
                         max_seq_length=MAX_SEQ_LENGTH)

In [ ]:
# split dataset into train and validation and get a loader
train_loader, val_loader = get_LSTM_dataloaders(lstm_dataset, test_ratio=0.1)

In [ ]:
# set model parameters
vocab_size = len(lstm_dataset.token2idx)
input_dim = 100
hidden_dim = 100
dropout_rate = 0.15

In [ ]:
model = models.LSTM(vocab_size=vocab_size,input_dim=input_dim,hidden_dim=hidden_dim,dropout_rate=dropout_rate)

In [ ]:
# ignore pad index when computing cross entropy
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=lstm_dataset.pad_idx)

In [ ]:
# save initial state dict of the model
torch.save(model.state_dict(), 'models/LSTM.pt')

Restart here for evaluating multiple optimizers

In [ ]:
# load the state dict to restart from base model
model.load_state_dict(torch.load('models/LSTM.pt'))

In [ ]:
# choose optimizer for this run
optimizer = torch.optim.RMSprop(model.parameters())

In [ ]:
manager = train_manager(model, loss_fn, optimizer, train_loader, val_loader, device=device)

In [ ]:
# train the model and return train loss and validation accuracy
losses = manager.train(20, verbose=True, eval_all_epochs=True, eval_mode='accuracy', is_LSTM=True)

In [ ]:
# get max validation accuracy
print(torch.max(torch.tensor(losses[1])))

In [ ]:
torch.save(torch.tensor(losses[1]), 'results/RMSprop_P5.pt')